# Описание
Данный ноутбук выполняет полный цикл обработки данных для иерархической классификации. 

Он включает:
 - предварительную очистку и преобразование текстовых и табличных признаков;
 - извлечение структурированной информации из неформализованных атрибутов;
 - генерацию текстовых эмбеддингов на основе BERT-модели;
 - предсказание категорий с помощью каскада иерархических моделей (level_1 → level_5) с учётом терминальных классов.
 
В результате формируется финальный DataFrame с предсказанными категориями (predicted_cat) для каждого объекта.

# Import

In [1]:
import pandas as pd
import numpy as np
import json
import re
import copy

import torch
from transformers import AutoTokenizer, AutoModel

from catboost import Pool

import argparse
import pickle

from tqdm.auto import tqdm
tqdm.pandas(desc='Tokenizing rows')

import warnings
warnings.filterwarnings('ignore')

RAND = 42

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
model_name = 'cointegrated/rubert-tiny2'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(83828, 312, padding_idx=0)
    (position_embeddings): Embedding(2048, 312)
    (token_type_embeddings): Embedding(2, 312)
    (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-2): 3 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=312, out_features=312, bias=True)
            (key): Linear(in_features=312, out_features=312, bias=True)
            (value): Linear(in_features=312, out_features=312, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=312, out_features=312, bias=True)
            (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)

In [ ]:
with open('terminators.pkl', 'rb') as f:
    terminators = pickle.load(f)

with open('clf_cat_1.pkl', 'rb') as f:
    model_level_1 = pickle.load(f)

with open('clf_cat_level_2.pkl', 'rb') as f:
    model_level_2 = pickle.load(f)

with open('clf_cat_level_3.pkl', 'rb') as f:
    model_level_3 = pickle.load(f)

with open('clf_cat_level_4.pkl', 'rb') as f:
    model_level_4 = pickle.load(f)

with open('clf_cat_level_5.pkl', 'rb') as f:
    model_level_5 = pickle.load(f)

# Function

In [ ]:
def attributes_to_text(attr_json) -> str:
    """
    Извлекает атрибуты из json в строку.
    """
    try:
        attributes = json.loads(attr_json.replace('""', '"'))

        attr_texts = [
            f"{attr['attribute_name']}: {attr['attribute_value']}"
            for attr in attributes if attr['attribute_name'] not in
            ['Название', 'В наличии', 'Ebsmstock']
        ]

        return '. '.join(attr_texts) if attr_texts else 'нет'

    except Exception as e:
        return 'Атрибуты отсутствуют'

In [ ]:
def cleaned_text(text) -> str:
    """
    Простая очистка текста.
    """
    text = str(
        text
    ) if text is not None else ''  # преобразуем text в строку, если это не строка
    text = text.lower()
    text = re.sub(r'[^а-яёa-z0-9\s.,*!?:-]', '',
                  text)  # удаление лишних символов (кроме пунктуации)
    text = re.sub(r'\s+', ' ', text).strip()  # удаление лишних пробелов
    return text

In [ ]:
def extract_info(attr_str: str) -> tuple:
    """
    Извлекает структурированные данные из текстовой строки с атрибутами товара, 
    представленной в виде одного объединённого текста. Возвращает кортеж 
    с основными характеристиками товара.
    
    :param attr_str: cтрока с описанием атрибутов товара
    :return: кортеж из элементов.
    """
    # преобразуем attr_str в строку, если это не строка
    attr_str = str(attr_str) if attr_str is not None else ''

    # извлекаем поставщика
    supplier_match = re.search(r'поставщик:\s*([^\.]+)', attr_str)
    if supplier_match:
        supplier = supplier_match.group(1).strip()
        if supplier.lower() == "нет бренда":  # если поставщик - нет бренда
            supplier = None
    else:
        supplier = None

    # извлекаем модель устройства
    model_match = re.search(r'модель устройства:\s*([^\.]+)', attr_str)
    model = model_match.group(1).strip() if model_match else None

    # извлекаем страну
    country_match = re.search(r'страна:\s*([^\.]+)', attr_str)
    country = country_match.group(1).strip() if country_match else None

    # извлекаем материал
    material_match = re.search(r'материал:\s*([^\.]+)', attr_str)
    material = material_match.group(1).strip() if material_match else None

    # извлекаем самовывоз (1 = да, 0 = нет)
    pickup_match = re.search(r'возможность самовывоза:\s*(да|нет)', attr_str)
    pickup = 1 if pickup_match and pickup_match.group(1) == 'да' else 0

    # извлекаем доставку (1 = да, 0 = нет)
    delivery_match = re.search(r'возможность доставки:\s*(да|нет)', attr_str)
    delivery = 1 if delivery_match and delivery_match.group(1) == 'да' else 0

    # извлекаем гарантию (1 = да, 0 = нет)
    guarantee_match = re.search(r'гарантия:\s*(да|нет)', attr_str)
    guarantee = 1 if guarantee_match and guarantee_match.group(
        1) == 'да' else 0

    # извлекаем вес
    weight_match = re.search(r'вес:\s*([^\.]+)', attr_str)
    if weight_match:
        weight_str = weight_match.group(1).strip()
        # убираем запятую и пробелы, если они есть
        weight_str = weight_str.replace(',', '.').replace(' ', '')
        try:
            weight = float(weight_str)
        except ValueError:
            weight = None  # если преобразование не удалось, устанавливаем None
    else:
        weight = None

    # извлекаем габариты
    size_match = re.search(
        r'размер: длина\s*(\d+)\s*ширина\s*(\d+)\s*высота\s*(\d+)', attr_str)
    if size_match:
        length, width, height = map(int, size_match.groups())
    else:
        length, width, height = None, None, None  # если измерения не найдены

    # извлекаем габариты упаковки
    size_match_p = re.search(
        r'ширина упаковки:\s*(\d+)\.\s*высота упаковки:\s*(\d+)\.\s*глубина упаковки:\s*(\d+)',
        attr_str)
    if size_match_p:
        length_p, width_p, height_p = map(int, size_match_p.groups())
    else:
        length_p, width_p, height_p = None, None, None  # если измерения не найдены

    # удаляем обработанные атрибуты из строки
    cleaned_attrs = re.sub(r'поставщик:\s*[^\.]+\.?', '', attr_str)
    cleaned_attrs = re.sub(r'возможность самовывоза:\s*(да|нет)\.?', '',
                           cleaned_attrs)
    cleaned_attrs = re.sub(r'страна:\s*[^\.]+\.?', '', cleaned_attrs)
    cleaned_attrs = re.sub(r'материал:\s*[^\.]+\.?', '', cleaned_attrs)
    cleaned_attrs = re.sub(r'модель устройства:\s*[^\.]+\.?', '',
                           cleaned_attrs)
    cleaned_attrs = re.sub(r'возможность доставки:\s*(да|нет)\.?', '',
                           cleaned_attrs)
    cleaned_attrs = re.sub(r'гарантия:\s*(да|нет)\.?', '', cleaned_attrs)
    cleaned_attrs = re.sub(r'вес\s*\d+\.?', '', cleaned_attrs)
    cleaned_attrs = re.sub(
        r'размер: длина\s*\d+\s*ширина\s*\d+\s*высота\s*\d+\.?', '',
        cleaned_attrs)
    cleaned_attrs = re.sub(
        r'ширина упаковки\s*\d+\s*высота упаковки\s*\d+\s*глубина упаковки\s*\d+\.?',
        '', cleaned_attrs)

    return supplier, country, material, model, pickup, length, width, height, delivery, guarantee, weight, length_p, width_p, height_p

In [ ]:
def get_bert_embedding(text: str) -> np.ndarray:
    """
    Получает эмбеддинг текста с использованием mean pooling 
    по всем токенам (кроме паддинга).
    
    :param text: входной текст.
    :return: усреднённый эмбеддинг по всем токенам.
    """
    tokens = tokenizer(text,
                       return_tensors='pt',
                       truncation=True,
                       padding='max_length',
                       max_length=27)

    # переносим данные на GPU, если он есть
    tokens = {key: val.to(device) for key, val in tokens.items()}

    with torch.no_grad():  # выключаем градиенты
        output = model(**tokens)
        last_hidden_state = output.last_hidden_state  # [batch_size, seq_len, hidden_dim]
        attention_mask = tokens['attention_mask']  # [batch_size, seq_len]

        # применяем attention mask: обнуляем эмбеддинги паддингов
        mask = attention_mask.unsqueeze(-1).expand(last_hidden_state.size())
        masked_embeddings = last_hidden_state * mask

        # усреднение по непаддинговым токенам
        summed = masked_embeddings.sum(dim=1)
        counts = mask.sum(dim=1)  # число непаддинговых токенов
        mean_pooled = summed / counts

    return mean_pooled.cpu().numpy().squeeze()

# Preprocessing

In [ ]:
def preprocessing(dataset: pd.DataFrame) -> pd.DataFrame:
    """
    Функция обрабатывает входной датасет, очищая и преобразуя текстовые и 
    категориальные признаки, извлекая информацию из текстов, 
    удаляя ненужные или малозначимые столбцы, и превращает текстовые данные 
    в эмбеддинги с помощью модели BERT. 
    Результатом является DataFrame, содержащий эмбеддинги и 
    полезные бинарные признаки.

    :param dataset: исходный DataFrame.
    :return df_embeddings: новый DataFrame, в котором каждый объект представлен 
                        эмбеддингом текста и дополнительными бинарными признаками.
    """
    labeled_train = copy.copy(dataset)
    #labeled_train.drop('hash_id', axis=1, inplace=True)

    # извлекаем атрибуты из json в строку
    labeled_train['attributes'] = labeled_train['attributes'].apply(
        attributes_to_text)
    # чистим текст
    labeled_train['attributes'] = labeled_train['attributes'].apply(
        cleaned_text)
    labeled_train['source_name'] = labeled_train['source_name'].apply(
        cleaned_text)

    # извлекаем данные из атрибутов
    labeled_train[[
        'поставщик', 'страна', 'материал', 'модель', 'самовывоз', 'длина',
        'ширина', 'высота', 'возможность доставки', 'гарантия', 'вес',
        'длина_уп', 'ширина_уп', 'высота_уп'
    ]] = labeled_train['attributes'].progress_apply(
        lambda x: pd.Series(extract_info(x)))

    # удаляем старый столбец
    labeled_train.drop(columns=['attributes'], inplace=True)
    labeled_train.reset_index(drop=True, inplace=True)

    # объединяем source_name и модель в новый столбец source_name_model и удаляем их
    labeled_train['source_name_model'] = labeled_train.apply(
        lambda row:
        f"{row['source_name']}{'.' if row['модель'] is not None else ''} {row['модель'] or ''}"
        .strip(),
        axis=1)
    labeled_train.drop(['source_name', 'модель'], axis=1, inplace=True)

    # удаляем остальные столбцы где много пропусков
    labeled_train.drop([
        'поставщик', 'страна', 'длина', 'ширина', 'высота', 'вес', 'материал',
        'длина_уп', 'ширина_уп', 'высота_уп'
    ],
                       axis=1,
                       inplace=True)

    # преобразовываем данные
    for i in labeled_train.columns:
        if labeled_train[i].dtype == 'object':
            labeled_train[i] = labeled_train[i].astype('category')
        if i in ('самовывоз', 'возможность доставки', 'гарантия'):
            labeled_train[i] = labeled_train[i].astype('int8')

    # преобразуем тексты в эмбеддинги
    texts = labeled_train['source_name_model'].tolist()
    #labels = labeled_train['label'].values  # метки классов
    pickup = labeled_train['самовывоз'].values
    delivery = labeled_train['возможность доставки'].values
    guarantee = labeled_train['гарантия'].values

    # преобразуем все тексты в эмбеддинги
    embeddings = np.array([get_bert_embedding(text) for text in tqdm(texts)])

    # преобразовываем матрицу в DataFrame
    df_embeddings = pd.DataFrame(embeddings)
    df_embeddings['pickup'] = pickup
    df_embeddings['delivery'] = delivery
    df_embeddings['guarantee'] = guarantee

    return df_embeddings

# Inference

In [ ]:
def levels_predict(data: pd.DataFrame, 
                   hash_data: pd.DataFrame) -> pd.DataFrame:
    """
    Функция выполняет многоуровневую (иерархическую) классификацию объектов 
    с использованием последовательных моделей для каждого уровня иерархии категорий. 
    В результате функция возвращает предсказанную финальную категорию (predicted_cat) 
    для каждого объекта.
    
    :param data: таблица признаков объектов для инференса. 
    :param hash_data: таблица с идентификаторами hash_id, соответствующими объектам из data. 
    :return output: таблица с предсказаными категориями
    """
    # level_1
    pool = Pool(data)
    y_pred = model_level_1.predict(pool,
                                   prediction_type='Class',
                                   ntree_start=0,
                                   ntree_end=0,
                                   thread_count=-1,
                                   verbose=None,
                                   task_type="CPU")

    data['level_1'] = y_pred

    # level_2
    pool2 = Pool(data.drop(['level_1'], axis=1))
    y_pred2 = model_level_2.predict(pool2,
                                    prediction_type='Class',
                                    ntree_start=0,
                                    ntree_end=0,
                                    thread_count=-1,
                                    verbose=None,
                                    task_type='CPU')

    data['level_2'] = y_pred2
    data['hash_id'] = hash_data['hash_id']

    # конечные категории для второго уровня
    level_2_term = data[data['level_2'].isin(terminators)][[
        'level_2', 'hash_id'
    ]]
    level_2_term.rename(columns={'level_2': 'predicted_cat'}, inplace=True)

    # level_3
    for_level_3 = data[(data['level_2'].isin(terminators) == False)]
    pool3 = Pool(for_level_3.drop(['hash_id', 'level_1', 'level_2'], axis=1))
    y_pred3 = model_level_3.predict(pool3,
                                    prediction_type='Class',
                                    ntree_start=0,
                                    ntree_end=0,
                                    thread_count=-1,
                                    verbose=None,
                                    task_type='CPU')

    level_3_res = for_level_3
    level_3_res['level_3'] = y_pred3

    # конечные категории для третьего уровня
    level_3_term = level_3_res[level_3_res['level_3'].isin(terminators)][[
        'level_3', 'hash_id'
    ]]
    level_3_term.rename(columns={'level_3': 'predicted_cat'}, inplace=True)

    # level_4
    for_level_4 = level_3_res[(
        level_3_res['level_3'].isin(terminators) == False)]
    pool4 = Pool(
        for_level_4.drop(['hash_id', 'level_1', 'level_2', 'level_3'], axis=1))
    y_pred4 = model_level_4.predict(pool4,
                                    prediction_type='Class',
                                    ntree_start=0,
                                    ntree_end=0,
                                    thread_count=-1,
                                    verbose=None,
                                    task_type='CPU')

    level_4_res = for_level_4
    level_4_res['level_4'] = y_pred4

    # конечные категории для четвертого уровня
    level_4_term = level_4_res[level_4_res['level_4'].isin(terminators)][[
        'level_4', 'hash_id'
    ]]
    level_4_term.rename(columns={'level_4': 'predicted_cat'}, inplace=True)

    #level_5
    for_level_5 = level_4_res[(
        level_4_res['level_4'].isin(terminators) == False)]
    pool5 = Pool(
        for_level_5.drop(
            ['hash_id', 'level_1', 'level_2', 'level_3', 'level_4'], axis=1))
    y_pred5 = model_level_5.predict(pool5,
                                    prediction_type='Class',
                                    ntree_start=0,
                                    ntree_end=0,
                                    thread_count=-1,
                                    verbose=None,
                                    task_type='CPU')

    level_5_res = for_level_5
    level_5_res['level_5'] = y_pred5

    # конечные категории для пятого уровня
    level_5_term = level_5_res[level_5_res['level_5'].isin(terminators)][[
        'level_5', 'hash_id'
    ]]
    level_5_term.rename(columns={'level_5': 'predicted_cat'}, inplace=True)

    # объединяем предсказания по уровням в один датасет
    level_23 = pd.concat([level_2_term, level_3_term])
    level_234 = pd.concat([level_23, level_4_term])
    full_level = pd.concat([level_234, level_5_term])

    # сортируем значения
    full_level.sort_index(axis=0, inplace=True)

    output = pd.DataFrame()
    output['hash_id'] = full_level['hash_id']
    output['predicted_cat'] = full_level['predicted_cat'].astype(int)

    return output